# Создание и заполнение данных БД Postgre

In [1]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor


In [2]:
POSTGRESQL_HOST = '10.129.0.25'

In [3]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


In [4]:
!psql -c 'create database test;'

CREATE DATABASE


In [5]:
%load_ext sql

In [6]:
%%sql postgresql:///test
    SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

0 rows affected.


table


## Примеры SQL запросов 
1. Представим запрос для создания таблицы users c двумя полями:
```
%%sql postgresql:///test
    CREATE TABLE users (
        login varchar(10) NOT NULL, 
        password varchar(100)
    );
```

2. Чтобы добавить запись в таблицу, небходимо выполнить следующий запрос:

```
%%sql postgresql:///test
   INSERT INTO users VALUES ('monit', 'querty')
```

3. Получить данные из таблицы:
```
%%sql postgresql:///test
   SELECT * FROM users
```

4. Удалить таблицу:
```
!psql -d test -c 'DROP TABLE users'
```

Считаем данные из файла datasets/aggrigation_logs_per_week.csv.

Выведем первую строку с заголовками. 

In [7]:
import csv

with open('datasets/aggrigation_logs_per_week.csv') as file:
    reader = csv.reader(file)
    headers = next(reader)
print (headers)

['courseid', 'userid', 'num_week', 's_all', 's_all_avg', 's_course_viewed', 's_course_viewed_avg', 's_q_attempt_viewed', 's_q_attempt_viewed_avg', 's_a_course_module_viewed', 's_a_course_module_viewed_avg', 's_a_submission_status_viewed', 's_a_submission_status_viewed_avg', 'NameR_Level', 'Name_vAtt', 'Depart', 'Name_OsnO', 'Name_FormOPril', 'LevelEd', 'Num_Sem', 'Kurs', 'Date_vAtt']


Вам предоставлен набор данных с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу.
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Создадим таблицу с логами студентов в тестовой БД

In [8]:
!psql -d test -c 'CREATE TABLE logs ( \
    courseid varchar(5), \
    userid varchar(5), \
    num_week smallint, \
    s_all smallint, \
    s_all_avg varchar(10), \
    s_course_viewed smallint, \
    s_course_viewed_avg varchar(10), \
    s_q_attempt_viewed smallint, \
    s_q_attempt_viewed_avg varchar(10), \
    s_a_course_module_viewed smallint, \
    s_a_course_module_viewed_avg varchar(10), \
    s_a_submission_status_viewed smallint, \
    s_a_submission_status_viewed_avg varchar(10), \
    NameR_Level varchar(20), \
    Name_vAtt varchar(20), \
    Depart varchar(5), \
    Name_OsnO smallint, \
    Name_FormOPril smallint, \
    LevelEd smallint, \
    Num_Sem smallint, \
    Kurs smallint, \
    Date_vAtt varchar(20)\
);'

CREATE TABLE


Добавим данные из файла aggrigation_logs_per_week.csv в созданную таблицу.


In [9]:
!psql -d test \
    -c "\copy logs FROM 'datasets/aggrigation_logs_per_week.csv' DELIMITER ',' CSV HEADER;"

COPY 414528


Выведем первые 10 записей.

In [10]:
%%sql postgresql:///test
    SELECT * FROM logs LIMIT 10

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71262,34527,6,9,9,4,4,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,7,0,"4,5",0,2,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,8,0,3,0,"1,3333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,9,0,"2,25",0,1,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,10,0,"1,8",0,"0,8",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,11,1,"1,6667",1,"0,8333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,12,0,"1,4286",0,"0,7143",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,13,0,"1,25",0,"0,625",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,14,0,"1,1111",0,"0,5556",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,15,1,"1,1",1,"0,6",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022


## Задание 1:

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 

У какой кафедры больше всего курсов на портале?

## Задание 4:

Имеется информация о кодировании каждой кафедры:

| Id  | Name     | Id  | Name     | Id  | Name     | Id  | Name     |
|-----|---------|-----|---------|-----|---------|-----|---------|
| 1   | АиИИ    | 12  | ИиИБ    | 23  | ПОиД    | 34  | УСиБА    |
| 2   | АСУ     | 13  | ИТМ     | 24  | Психол. | 35  | Физики   |
| 3   | АЭПиМ   | 14  | ЛиП     | 25  | ПЭиБЖД  | 36  | Физкульт.|
| 4   | БИиИТ   | 15  | ЛиУТС   | 26  | РМПИ    | 37  | Химии    |
| 5   | ВИ      | 16  | ЛПиМ    | 27  | РЯОЯиМК | 38  | ХОМ      |
| 6   | ВТиП    | 17  | Менеджм.| 28  | СРиППО  | 39  | ЦДОМ     |
| 7   | ГМДиОПИ | 18  | МиТОДиМ | 29  | CC      | 40  | ЭиМЭ     |
| 8   | ГМиТТК  | 19  | МиХТ    | 30  | ТиЭС    | 41  | Эконом.  |
| 9   | ГМУиУП  | 20  | ПиСЗ    | 31  | ТОМ     | 42  | ЭПП      |
| 10  | Дизайна | 21  | ПиЭММО  | 32  | ТССА    | 43  | ЯиЛ      |
| 11  | ДиСО    | 22  | ПМиИ    | 33  | УиИС    |     |          |


Создайте отдельно таблицу со справочником (номер кафедры и название(аббревиатура) кафедры) departments.

Внесите данные в таблицу. 

## Задание 5:

Выполните задание 3, но теперь нужно вывести не код кафедры, а ее название и количество курсов, которые за ней закреплены. 

У какой кафедры больше всего курсов?

## Задание 6:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




## Задание 7:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


## Задание 6:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

## Задание 9:

Выведите по каждой недели среднее количество всех событий на портале.

## Задание 10: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 